In [57]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns, stock_market_simulation
import xgboost as xgb
import joblib


specific_model_decisions = pd.read_csv('data/specific_model_decisions.csv').sort_values(by="Day").sort_values(by='Stock Name')
general_model_decisions = pd.read_csv('data/general_model_decisions.csv').sort_values(by="Day").sort_values(by='Stock Name')
# general_model_decisions.to_csv('data/general_model_decisions.csv', index=False)
# specific_model_decisions.to_csv('data/specific_model_decisions.csv', index=False)




In [56]:
general_model_decisions[general_model_decisions['Portfolio Value'] > 10000]

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date


In [51]:
specific_model_decisions[(specific_model_decisions['Portfolio Value'] == 10055.505004882812)]

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
1131,TDG,2.0,Buy,1400.359985,5854.425049,3.0,10055.505005,2024-10-11


In [54]:
specific_model_decisions[specific_model_decisions['Stock Name'] == 'ZBH']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
500,ZBH,0.0,Buy,103.000000,9897.000000,1.0,10000.000000,2024-09-10
1356,ZBH,2.0,Buy,103.894997,9690.285004,3.0,10001.969994,2024-10-11
854,ZBH,1.0,Buy,102.820000,9794.180000,2.0,9999.820000,2024-10-10
